# Boolean pancreatic cancer

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

import scripts.boolean_helper
from scripts.grid_class import GridParms
from scripts.tree_class import Tree, plotReactionGraph
from scripts.notebooks.output_helper import *

reaction_system = scripts.boolean_helper.convertRulesToReactions("scripts/models/boolean_rulefiles/pancreatic_cancer.hpp")

plt.style.use("./scripts/notebooks/custom_style_boolean.mplstyle")
%matplotlib inline

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
seed = 5

## Entropy, cuts and graphs

In [ ]:
# Best
tree = readTree("output/pancreatic_pbn_r5_e_tau1e-2/output_t0.nc")
r_out = np.ones(tree.n_internal_nodes, dtype="int") * 5
tree.initialize(reaction_system, r_out)
 
plotReactionGraph(tree.G, "plots/pancreatic_graph_best.pdf", colors[2], seed=seed)
printEntropyCuts(tree)

In [ ]:
# Reasonable
tree = readTree("output/pancreatic_prn_r5_e_tau1e-2/output_t0.nc")
r_out = np.ones(tree.n_internal_nodes, dtype="int") * 5
tree.initialize(reaction_system, r_out)

plotReactionGraph(tree.G, "plots/pancreatic_graph_reasonable.pdf", colors[0], seed=seed)
printEntropyCuts(tree)

In [ ]:
# Worst
tree = readTree("output/pancreatic_pwn_r5_e_tau1e-2/output_t0.nc")
r_out = np.ones(tree.n_internal_nodes, dtype="int") * 5
tree.initialize(reaction_system, r_out)

plotReactionGraph(tree.G, "plots/pancreatic_graph_worst.pdf", colors[1], seed=seed)
printEntropyCuts(tree)

## Load initial data

### Best

In [ ]:
time_series = TimeSeries("output/pancreatic_pbn_r5_e_tau1e-2")
time = time_series.time
moments_pb_r5 = time_series.calculateMoments()

In [ ]:
time_series = TimeSeries("output/pancreatic_pbn_r10_e_tau1e-2")
time = time_series.time
moments_pb_r10 = time_series.calculateMoments()

In [ ]:
time_series = TimeSeries("output/pancreatic_pbn_r20_e_tau1e-2")
time = time_series.time
moments_pb_r20 = time_series.calculateMoments()

### Reasonable

In [ ]:
time_series = TimeSeries("output/pancreatic_prn_r5_e_tau1e-2")
time = time_series.time
moments_pr_r5 = time_series.calculateMoments()

In [ ]:
time_series = TimeSeries("output/pancreatic_prn_r10_e_tau1e-2")
time = time_series.time
moments_pr_r10 = time_series.calculateMoments()

In [ ]:
time_series = TimeSeries("output/pancreatic_prn_r20_e_tau1e-2")
time = time_series.time
moments_pr_r20 = time_series.calculateMoments()

### Worst

In [ ]:
time_series = TimeSeries("output/pancreatic_pwn_r5_e_tau1e-2")
time = time_series.time
moments_pw_r5 = time_series.calculateMoments()

In [ ]:
time_series = TimeSeries("output/pancreatic_pwn_r10_e_tau1e-2")
time = time_series.time
moments_pw_r10 = time_series.calculateMoments()

In [ ]:
time_series = TimeSeries("output/pancreatic_pwn_r20_e_tau1e-2")
time = time_series.time
moments_pw_r20 = time_series.calculateMoments()

### Reference solution

In [ ]:
# reference = np.load("scripts/reference_solutions/pancreatic_r50.npy")

time_series = TimeSeries("output/pancreatic_pbn_r50_e_tau5e-3")
time = time_series.time
reference = time_series.calculateMoments()

In [ ]:
time_series = TimeSeries("output/pancreatic_pbn_r60_e_tau5e-3")
time = time_series.time
reference_r60 = time_series.calculateMoments()

## Plots

In [ ]:
names = time_series.getSpeciesNames()
names_lower = np.array([name.lower() for name in names])
names = names[np.argsort(names_lower)]

In [ ]:
fig, axs = plt.subplots(6, 6, figsize=(15, 15))

for i, ax in enumerate(axs.flatten()):
    if i < 34:
        ax.plot(time, reference[0][names[i]]-reference_r60[0][names[i]], ".-")
        ax.set_title(names[i])
axs[-1, -2].axis("off")
axs[-1, -1].axis("off")
plt.tight_layout()
plt.savefig("plots/pancreatic_reference_solution.pdf")

### Comparison for all species

In [ ]:
fig, axs = plt.subplots(6, 6, figsize=(15, 15))

for i, ax in enumerate(axs.flatten()):
    if i < 34:
        ax.plot(time, reference[0][names[i]], "k.-", label="reference")
        ax.plot(time, moments_pb_r5[0][names[i]], ":", color=colors[2], label="best, $r=5$")
        ax.plot(time, moments_pb_r10[0][names[i]], "--", color=colors[2], label="best, $r=10$")
        ax.plot(time, moments_pb_r20[0][names[i]], "-", color=colors[2], label="best, $r=20$")
        ax.plot(time, moments_pr_r5[0][names[i]], ":", color=colors[0], label="reasonable, $r=5$")
        ax.plot(time, moments_pr_r10[0][names[i]], "--", color=colors[0], label="reasonable, $r=10$")
        ax.plot(time, moments_pr_r20[0][names[i]], "-", color=colors[0], label="reasonable, $r=20$")
        ax.plot(time, moments_pw_r5[0][names[i]], ":", color=colors[1], label="worst, $r=5$")
        ax.plot(time, moments_pw_r10[0][names[i]], "--", color=colors[1], label="worst, $r=10$")
        ax.plot(time, moments_pw_r20[0][names[i]], "-", color=colors[1], label="worst, $r=20$")
        ax.set_title(names[i])
h, l = axs[0, 0].get_legend_handles_labels()
axs[-1, -2].legend(h[::3], ["reference", "best", "reasonable", "worst"], loc="center left")
axs[-1, -2].axis('off')
axs[-1, -1].axis('off')
plt.setp(axs, ylim=[-0.05, 1.05])
plt.tight_layout()
plt.savefig("plots/pancreatic_moments.pdf")

### Comparison for five relevant species

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(8, 5.5))
relevant_species = [1, 3, 20, 22, 25]
linestyles = [':', "--", "-"]

ax = axs.flatten()
for ax_i, i in enumerate(relevant_species):
    if i < 34:
        l0, = ax[ax_i].plot(time, reference_r60[0][names[i]], "k.-", label="reference")
        ax[ax_i].plot(time, moments_pb_r5[0][names[i]], ls=linestyles[0], color=colors[2], label="best, $r=5$")
        ax[ax_i].plot(time, moments_pb_r10[0][names[i]], ls=linestyles[1], color=colors[2], label="best, $r=10$")
        l1, = ax[ax_i].plot(time, moments_pb_r20[0][names[i]], ls=linestyles[2], color=colors[2], label="best, $r=20$")
        ax[ax_i].plot(time, moments_pr_r5[0][names[i]], ls=linestyles[0], color=colors[0], label="reasonable, $r=5$")
        ax[ax_i].plot(time, moments_pr_r10[0][names[i]], ls=linestyles[1], color=colors[0], label="reasonable, $r=10$")
        l2, = ax[ax_i].plot(time, moments_pr_r20[0][names[i]], ls=linestyles[2], color=colors[0], label="reasonable, $r=20$")
        ax[ax_i].plot(time, moments_pw_r5[0][names[i]], ls=linestyles[0], color=colors[1], label="worst, $r=5$")
        ax[ax_i].plot(time, moments_pw_r10[0][names[i]], ls=linestyles[1], color=colors[1], label="worst, $r=10$")
        l3, = ax[ax_i].plot(time, moments_pw_r20[0][names[i]], ls=linestyles[2], color=colors[1], label="worst, $r=20$")
        ax[ax_i].set_title(names[i])

dummy_lines = []
for line in linestyles:
    dummy_lines.append(axs[0, 0].plot([],[], ls=line, color="gray")[0])
legend1 = axs[-1, -1].legend([l0, l1, l2, l3], 
                             ["reference", "best", "reasonable", "worst"], 
                             loc="center left", 
                             borderpad=0, 
                             handletextpad=0.5,
                             borderaxespad=0
                             )
legend2 = axs[-1, -1].legend(dummy_lines, 
                             ["$r=5$", "$r=10$", "$r=20$"], 
                             loc="center right", 
                             borderpad=0, 
                             handletextpad=0.5,
                             borderaxespad=0
                             )
fig.add_artist(legend1)

axs[-1, -1].axis('off')
plt.setp(axs, xlabel="$t$")
plt.setp(axs[:, 0], ylabel="$\langle x \\rangle (t)$")
plt.tight_layout()
plt.savefig("plots/pancreatic_moments_relevant_species.pdf", bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(8, 6))
relevant_species = [1, 3, 20, 22, 25]
linestyles = [':', "--", "-"]

ax = axs.flatten()
for ax_i, i in enumerate(relevant_species):
    if i < 34:
        ax[ax_i].plot(time, np.abs(moments_pb_r5[0][names[i]]-reference[0][names[i]]), ls=linestyles[0], color=colors[2], label="best, $r=5$")
        ax[ax_i].plot(time, np.abs(moments_pb_r10[0][names[i]]-reference[0][names[i]]), ls=linestyles[1], color=colors[2], label="best, $r=10$")
        l0, = ax[ax_i].plot(time, np.abs(moments_pb_r20[0][names[i]]-reference[0][names[i]]), ls=linestyles[2], color=colors[2], label="best, $r=20$")
        ax[ax_i].plot(time, np.abs(moments_pr_r5[0][names[i]]-reference[0][names[i]]), ls=linestyles[0], color=colors[0], label="reasonable, $r=5$")
        ax[ax_i].plot(time, np.abs(moments_pr_r10[0][names[i]]-reference[0][names[i]]), ls=linestyles[1], color=colors[0], label="reasonable, $r=10$")
        l1, = ax[ax_i].plot(time, np.abs(moments_pr_r20[0][names[i]]-reference[0][names[i]]), ls=linestyles[2], color=colors[0], label="reasonable, $r=20$")
        ax[ax_i].plot(time, np.abs(moments_pw_r5[0][names[i]]-reference[0][names[i]]), ls=linestyles[0], color=colors[1], label="worst, $r=5$")
        ax[ax_i].plot(time, np.abs(moments_pw_r10[0][names[i]]-reference[0][names[i]]), ls=linestyles[1], color=colors[1], label="worst, $r=10$")
        l2, = ax[ax_i].plot(time, np.abs(moments_pw_r20[0][names[i]]-reference[0][names[i]]), ls=linestyles[2], color=colors[1], label="worst, $r=20$")
        ax[ax_i].set_title(names[i])

dummy_lines = []
for line in linestyles:
    dummy_lines.append(axs[0, 0].plot([],[], ls=line, color="gray")[0])
legend1 = axs[-1, -1].legend([l0, l1, l2],
                             ["best", "reasonable", "worst"], 
                             loc="center left", 
                             borderpad=0, 
                             handletextpad=0.5, 
                             borderaxespad=0
                             )
legend2 = axs[-1, -1].legend(dummy_lines, 
                             ["$r=5$", "$r=10$", "$r=20$"], 
                             loc="center right", 
                             borderpad=0, 
                             handletextpad=0.5, 
                             borderaxespad=0
                             )
fig.add_artist(legend1)

axs[-1, -1].axis('off')
plt.setp(axs, xlabel="$t$")
plt.setp(axs[:, 0], ylabel="$|\langle x \\rangle - \langle x_\mathrm{{ref}} \\rangle|$")
plt.tight_layout()
plt.savefig("plots/pancreatic_moments_relevant_species_err.pdf")

### Total error over all species

In [ ]:
def calculateAbsErr(marginal_distribution, ord=None):
    difference = np.zeros((len(names), len(time)))
    for i, name in enumerate(names):
        difference[i, :] = marginal_distribution[name] - reference_r60[0][name]
    abs_err = np.linalg.norm(difference, axis=0, ord=ord)
    return abs_err

In [ ]:
ord = np.inf

fig, axs = plt.subplots(figsize=(5, 4))

axs.plot(time, calculateAbsErr(moments_pb_r5[0], ord=ord), ":", color=colors[2], label="best, $r=5$")
axs.plot(time, calculateAbsErr(moments_pb_r10[0], ord=ord), "--", color=colors[2], label="best, $r=10$")
l0, = axs.plot(time, calculateAbsErr(moments_pb_r20[0], ord=ord), "-", color=colors[2], label="best, $r=20$")
axs.plot(time, calculateAbsErr(moments_pr_r5[0], ord=ord), ":", color=colors[0], label="reasonable, $r=5$")
axs.plot(time, calculateAbsErr(moments_pr_r10[0], ord=ord), "--", color=colors[0], label="reasonable, $r=10$")
l1, = axs.plot(time, calculateAbsErr(moments_pr_r20[0], ord=ord), "-", color=colors[0], label="reasonable, $r=20$")
axs.plot(time, calculateAbsErr(moments_pw_r5[0], ord=ord), ":", color=colors[1], label="worst, $r=5$")
axs.plot(time, calculateAbsErr(moments_pw_r10[0], ord=ord), "--", color=colors[1], label="worst, $r=10$")
l2, = axs.plot(time, calculateAbsErr(moments_pw_r20[0], ord=ord), "-", color=colors[1], label="worst, $r=20$")
axs.legend([l0, l1, l2], ["best", "reasonable", "worst"])
axs.set_xlabel("$t$")
axs.set_ylabel("$\| \langle x_i \\rangle - \langle x_{{i,\mathrm{{ref}}}} \\rangle \|_{{\infty,i}}$")
plt.tight_layout()
plt.savefig("plots/pancreatic_moments_err_infty.pdf")

In [ ]:
ord = np.inf

fig, axs = plt.subplots(1, 3, figsize=(7, 2.3))

axs[0].plot(time, calculateAbsErr(moments_pb_r5[0], ord=ord), "-", color=colors[2], label="best, $r=5$")
axs[1].plot(time, calculateAbsErr(moments_pb_r10[0], ord=ord), "-", color=colors[2], label="best, $r=10$")
l0, = axs[2].plot(time, calculateAbsErr(moments_pb_r20[0], ord=ord), "-", color=colors[2], label="best, $r=20$")
axs[0].plot(time, calculateAbsErr(moments_pr_r5[0], ord=ord), "-", color=colors[0], label="reasonable, $r=5$")
axs[1].plot(time, calculateAbsErr(moments_pr_r10[0], ord=ord), "-", color=colors[0], label="reasonable, $r=10$")
l1, = axs[2].plot(time, calculateAbsErr(moments_pr_r20[0], ord=ord), "-", color=colors[0], label="reasonable, $r=20$")
axs[0].plot(time, calculateAbsErr(moments_pw_r5[0], ord=ord), "-", color=colors[1], label="worst, $r=5$")
axs[1].plot(time, calculateAbsErr(moments_pw_r10[0], ord=ord), "-", color=colors[1], label="worst, $r=10$")
l2, = axs[2].plot(time, calculateAbsErr(moments_pw_r20[0], ord=ord), "-", color=colors[1], label="worst, $r=20$")
fig.legend([l0, l1, l2], ["best", "reasonable", "worst"], ncols=3, loc="center", bbox_to_anchor=(0.5, 1.01))
plt.setp(axs, xlabel="$t$", ylim=[-0.01, 0.36], xticks=[0, 5, 10, 15, 20])
axs[0].set_ylabel("$\| \langle x_i \\rangle (t) - \langle x_{{i,\mathrm{{ref}}}} \\rangle (t) \|_{{\infty,i}}$")

axs[0].set_title("$r=5$")
axs[1].set_title("$r=10$")
axs[2].set_title("$r=20$")

axs[1].set_yticklabels([])

axs[2].yaxis.tick_right()
axs[2].yaxis.set_ticks_position("both")
axs[2].yaxis.set_label_position("right")

plt.subplots_adjust(wspace=0.03)
plt.savefig("plots/pancreatic_moments_err_infty_separate.pdf", bbox_inches="tight")

In [ ]:
ord = None

fig, axs = plt.subplots(figsize=(5, 4))

axs.plot(time, calculateAbsErr(moments_pb_r5[0], ord=ord), ":", color=colors[2], label="best, $r=5$")
axs.plot(time, calculateAbsErr(moments_pb_r10[0], ord=ord), "--", color=colors[2], label="best, $r=10$")
l0, = axs.plot(time, calculateAbsErr(moments_pb_r20[0], ord=ord), "-", color=colors[2], label="best, $r=20$")
axs.plot(time, calculateAbsErr(moments_pr_r5[0], ord=ord), ":", color=colors[0], label="reasonable, $r=5$")
axs.plot(time, calculateAbsErr(moments_pr_r10[0], ord=ord), "--", color=colors[0], label="reasonable, $r=10$")
l1, = axs.plot(time, calculateAbsErr(moments_pr_r20[0], ord=ord), "-", color=colors[0], label="reasonable, $r=20$")
axs.plot(time, calculateAbsErr(moments_pw_r5[0], ord=ord), ":", color=colors[1], label="worst, $r=5$")
axs.plot(time, calculateAbsErr(moments_pw_r10[0], ord=ord), "--", color=colors[1], label="worst, $r=10$")
l2, = axs.plot(time, calculateAbsErr(moments_pw_r20[0], ord=ord), "-", color=colors[1], label="worst, $r=20$")
axs.legend([l0, l1, l2], ["best", "reasonable", "worst"])
axs.set_xlabel("$t$")
axs.set_ylabel("$\| \langle x_i \\rangle - \langle x_{{i,\mathrm{{ref}}}} \\rangle \|_{{2,i}}$")
plt.tight_layout()
plt.savefig("plots/pancreatic_moments_err_2.pdf")

In [ ]:
ord = None

fig, axs = plt.subplots(1, 3, figsize=(7, 2.3))

axs[0].plot(time, calculateAbsErr(moments_pb_r5[0], ord=ord), "-", color=colors[2], label="best, $r=5$")
axs[1].plot(time, calculateAbsErr(moments_pb_r10[0], ord=ord), "-", color=colors[2], label="best, $r=10$")
l0, = axs[2].plot(time, calculateAbsErr(moments_pb_r20[0], ord=ord), "-", color=colors[2], label="best, $r=20$")
axs[0].plot(time, calculateAbsErr(moments_pr_r5[0], ord=ord), "-", color=colors[0], label="reasonable, $r=5$")
axs[1].plot(time, calculateAbsErr(moments_pr_r10[0], ord=ord), "-", color=colors[0], label="reasonable, $r=10$")
l1, = axs[2].plot(time, calculateAbsErr(moments_pr_r20[0], ord=ord), "-", color=colors[0], label="reasonable, $r=20$")
axs[0].plot(time, calculateAbsErr(moments_pw_r5[0], ord=ord), "-", color=colors[1], label="worst, $r=5$")
axs[1].plot(time, calculateAbsErr(moments_pw_r10[0], ord=ord), "-", color=colors[1], label="worst, $r=10$")
l2, = axs[2].plot(time, calculateAbsErr(moments_pw_r20[0], ord=ord), "-", color=colors[1], label="worst, $r=20$")
fig.legend([l0, l1, l2], ["best", "reasonable", "worst"], ncols=3, loc="center", bbox_to_anchor=(0.5, 1.01))
plt.setp(axs, xlabel="$t$", ylim=[-0.03, 0.63], xticks=[0, 5, 10, 15, 20])
axs[0].set_ylabel("$\| \langle x_i \\rangle - \langle x_{{i,\mathrm{{ref}}}} \\rangle \|_{{2,i}}$")

axs[0].set_title("$r=5$")
axs[1].set_title("$r=10$")
axs[2].set_title("$r=20$")

axs[1].set_yticklabels([])

axs[2].yaxis.tick_right()
axs[2].yaxis.set_ticks_position("both")
axs[2].yaxis.set_label_position("right")

plt.subplots_adjust(wspace=0.03)
plt.savefig("plots/pancreatic_moments_err_2_separate.pdf", bbox_inches="tight")